<a href="https://colab.research.google.com/github/soichi-fujiwara/jupyter-notebook/blob/master/wikiWord2VecCreGroup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##学習済みDoc2Vecモデル
https://yag-ays.github.io/project/pretrained_doc2vec_wikipedia/

ライブラリ
Gensim: 3.6.0

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
from gensim.models import KeyedVectors
from pprint import pprint

#Gドライブ保存後の確認
model_dir = './drive/My Drive/NLP/wiki_tohoku.model'
model = KeyedVectors.load(model_dir)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [3]:
!pip install mecab-python3

     |████████████████████████████████| 15.9MB 40.7MB/s 


In [4]:
import pandas as pd
import itertools
#**************************************************************************
#分類特定
#**************************************************************************
max_cate = ""

is_a_parent = pd.read_table('./drive/My Drive/NLP/is_a_parent_list.txt')

#dataframeをlist化
is_a_parent_list = is_a_parent.values.tolist()
#2次元listから1次元listへ
is_a_parent_list = list(itertools.chain.from_iterable(is_a_parent_list))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  


In [0]:
import pandas as pd

def group_check(child_wd):

  ave_val_list = []
  ave_val_list.append(child_wd)
  
  #is-a関係の判別
  for is_a_word in is_a_parent_list:
    try:
      parent_wd = is_a_word
      isa_val_list = []

      for i in range(5):
        rvs_wd = model.most_similar(positive=[child_wd])[i][0]
        if child_wd != rvs_wd.replace('[',"").replace(']',""):
          wk_val = model.similarity(parent_wd,rvs_wd)
          isa_val_list.append(wk_val)

          for ii in range(5):
            rvs_wd2 = model.most_similar(positive=[rvs_wd])[ii][0]
            if rvs_wd != rvs_wd2.replace('[',"").replace(']',""):
              wk_val = model.similarity(parent_wd,rvs_wd2)
              isa_val_list.append(wk_val)

    except KeyError as error:
      print(error)

    ave_val = sum(isa_val_list) / len(isa_val_list)
    #max_val = max(isa_val_list)
    #min_val = min(isa_val_list)
    print("【",parent_wd,"】is-a:",ave_val)
    ave_val_list.append(ave_val)

  return ave_val_list

In [75]:
import MeCab
cnt = 0

del_list =['(',')','-','"','、','。','/','.',',',':',')。',')、','「','」','『','』','()','』(','=','～','+','']

tokenizer = MeCab.Tagger("-Ochasen")

for k, v in model.vocab.items():
  write_list = []

  if not k in del_list:
    node = tokenizer.parseToNode(k)
    while node:
      if node.feature.split(",")[0] == u"名詞":
        if(node.feature.split(",")[1] == u"サ変接続" or
           node.feature.split(",")[1] == u"一般" or
           node.feature.split(",")[1] == u"固有名詞"):
          #グループ判別
          rt_ave_val = group_check(k)
          #グループ判別値書き込み
          write_list.append(rt_ave_val)
          print(write_list)
          cnt = cnt + 1
      else:
        pass
      node = node.next

      if cnt > 5:
        break

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


【 男性 】is-a: 0.13246950536655883
【 女性 】is-a: 0.1667623377094666
【 キャラクター 】is-a: 0.20615097285287146
【 職業 】is-a: 0.20548359143237274
【 組織 】is-a: 0.1514538479425634
【 施設 】is-a: 0.16215323271850746
【 国家 】is-a: 0.1778785839676857
【 都市 】is-a: 0.10374845541179335
【 地域 】is-a: 0.1887072365731001
【 温泉 】is-a: 0.06706333421170711
【 自然 】is-a: 0.2672080773860216
【 天体 】is-a: 0.2311511122621596
【 道具 】is-a: 0.40936247805754344
【 材料 】is-a: 0.34149354298909507
【 芸術 】is-a: 0.07796574056070919
【 食べ物 】is-a: 0.33833018988370894
【 乗り物 】is-a: 0.28926198029269773
【 薬 】is-a: 0.26495711306730907
【 映画 】is-a: 0.11348534421995282
【 音楽 】is-a: 0.1002096096985042
【 番組 】is-a: 0.12392674167640508
【 家具 】is-a: 0.1557208184696113
【 本 】is-a: 0.15785353652512032
【 新聞 】is-a: 0.05900798061241706
【 ソフトウェア 】is-a: 0.248914592495809
【 絵画 】is-a: 0.18994430669893822
【 動物 】is-a: 0.29520407238354285
【 植物 】is-a: 0.23026406938831012
【 身体 】is-a: 0.2941113921503226
【 事件 】is-a: 0.19369404564301174
【 武器 】is-a: 0.29823676627129314
【 戦争 】is-a:

KeyboardInterrupt: ignored